In [5]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [6]:
## for Langsmith tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [32]:
## Data Ingestion - from the website we need to scrape the data 
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://datasciencedojo.com/blog/what-is-context-engineering/")
web_base_loader = loader.load()
web_base_loader

[Document(metadata={'source': 'https://datasciencedojo.com/blog/what-is-context-engineering/', 'title': 'What is Context Engineering? The New Foundation for Reliable AI and RAG Systems | Data Science Dojo', 'description': 'Context engineering is revolutionizing AI by enabling LLMs to deliver accurate, personalized, and enterprise-ready solutions. Discover the principles, techniques, and best practices for context engineering in this comprehensive guide.', 'language': 'en-US'}, page_content='\n\n\n\n\n\nWhat is Context Engineering? The New Foundation for Reliable AI and RAG Systems | Data Science Dojo\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFor a hands-on learning experience to develop LLM applications, join our LLM Bootcamp today.\n\nEarly Bird Discount Ending Soon!  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\nBootcamps \n\n\n\n\n\n\n\n\n\n\nWe offer 

In [33]:
#Dividing the document into chunks because the document is too long , llms have a limit on the number of tokens they can process
## Load data --> Docs --> Divide our text into chunks --> Vectors --> vector embeddings --> Store in vector database
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100) 
docs = text_splitter.split_documents(web_base_loader)

In [34]:
docs

[Document(metadata={'source': 'https://datasciencedojo.com/blog/what-is-context-engineering/', 'title': 'What is Context Engineering? The New Foundation for Reliable AI and RAG Systems | Data Science Dojo', 'description': 'Context engineering is revolutionizing AI by enabling LLMs to deliver accurate, personalized, and enterprise-ready solutions. Discover the principles, techniques, and best practices for context engineering in this comprehensive guide.', 'language': 'en-US'}, page_content='What is Context Engineering? The New Foundation for Reliable AI and RAG Systems | Data Science Dojo\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFor a hands-on learning experience to develop LLM applications, join our LLM Bootcamp today.\n\nEarly Bird Discount Ending Soon!  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\nBootcamps \n\n\n\n\n\n\n\n\n\n\nWe offer online and i

In [35]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [36]:
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_documents(docs, embeddings)

In [37]:
vector_store

In [38]:
query = "integrations have been split into lightweight packages that"
result = vector_store.similarity_search(query, k=3)
result[0].page_content

'Developer Platforms\nThe next evolution of coding assistants is moving beyond simple autocomplete. Systems are being built that have full context of an entire codebase, integrating with Language Server Protocols (LSP) to understand type errors, parsing production logs to identify bugs, and reading recent commits to maintain coding style. These agentic systems can autonomously write code, create pull requests, and even debug issues based on a rich, real-time understanding of the development environment.\nHyper-Personalization\nIn sectors like e-commerce, healthcare, and finance, deep context is enabling unprecedented levels of personalization. A financial advisor bot can provide tailored advice by accessing a user’s entire portfolio, their stated risk tolerance, and real-time market data. A healthcare assistant can offer more accurate guidance by considering a patient’s full medical history, recent lab results, and even data from wearable devices.\nBest Practices for Context Engineerin

In [39]:
#llms are used to generate the answer based on the context retrieved from the vector store
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001D76FFD6CB0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D76FFD5E70> root_client=<openai.OpenAI object at 0x000001D76FFD5300> root_async_client=<openai.AsyncOpenAI object at 0x000001D76FFD4640> model_name='gpt-4o-mini' model_kwargs={} openai_api_key=SecretStr('**********')


In [15]:
## Retrieval Chain 
## this is used to retrieve the relevant documents from the vector store based on the query
## context is the relevant documents retrieved from the vector store
## we pass the context to the llm to generate the answer
## this retrieval chain does this 
## previously we were using the vector store directly to get the relevant documents that match the query
## when we want to have a qna system we need to use a retrieval chain


In [41]:
# This is the retrieval chain that we will use to get the relevant documents from the vector store
#create_stuff_documents_chain is used to create a chain that takes the context and the query and returns the answer
#here the context is - the relevant documents retrieved from the vector store

from langchain.chains.combine_documents import create_stuff_documents_chain

# This is the prompt template that we will use to generate the answer
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context: 
<context>
{context}
</context>



"""
)

document_chain = create_stuff_documents_chain(llm,prompt)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context: \n<context>\n{context}\n</context>\n\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D76FFD6CB0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D76FFD5E70>, root_client=<openai.OpenAI object at 0x000001D76FFD5300>, root_async_client=<openai.AsyncOpenAI object at 0x000001D76FFD4640>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={

Retriever will dynamically select the most relevant documents or chunks and pass those in for a given question , these selected chunks are known as context


In [42]:
## Retriver - its an interface that allows us to retrieve the relevant documents from the vector store based on the query
## we convert this vector store into a retriever
retriever = vector_store.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain) # retriever is used to retrieve the relevant documents from the vector store based on the query and document_chain is used to generate the answer based on the context retrieved from the vector store
#the context is retrieved from the retriever and passed to the document_chain to generate the answer


In [43]:
response = retrieval_chain.invoke({
    "input" : "How RAG Works?"
})
response

{'input': 'How RAG Works?',
 'context': [Document(id='44499c3e-90d6-47b7-a621-210f3d0c5094', metadata={'source': 'https://datasciencedojo.com/blog/what-is-context-engineering/', 'title': 'What is Context Engineering? The New Foundation for Reliable AI and RAG Systems | Data Science Dojo', 'description': 'Context engineering is revolutionizing AI by enabling LLMs to deliver accurate, personalized, and enterprise-ready solutions. Discover the principles, techniques, and best practices for context engineering in this comprehensive guide.', 'language': 'en-US'}, page_content='3. Graph RAG\nFor structured enterprise data,\xa0Graph RAG\xa0retrieves interconnected entities and relationships from knowledge graphs, enabling multi-hop reasoning and richer, more accurate responses.\nLearn Advanced RAG Techniques in Large Language Models Bootcamp\nContext Engineering in Practice: Enterprise\nEnterprise Knowledge Federation\nEnterprises often struggle with knowledge fragmented across countless silo